In [1008]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

import flask
from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
from os import environ
import requests
import io

import datetime
from dateutil.relativedelta import relativedelta

import socket

import seaborn as sns
from itertools import groupby

In [1020]:
Data.to_csv('Data_forhome.csv')

In [1009]:
files_lg=[x for x in os.listdir() if '_lg_' in x and 'business' not in x]
cols_lg=['Origin','Destination','Carrier','Date','Time','drop','Yield','BidPrice','Customer','Direction','Interval']
Data_lg=DataFrame(columns=cols_lg)

for f in files_lg:

    Data_intermediate=pd.read_csv(f,header=None)

    Data_intermediate.columns=['Origin','Destination','Carrier','Date','Time','drop','Yield','BidPrice']

    Data_intermediate['Customer']=f.split('_')[4]
    Data_intermediate['Direction']=f.split('_')[5]
    Data_intermediate['Interval']=f.split('_')[6]
    Data_intermediate=Data_intermediate[cols_lg]

    Data_lg=pd.concat([Data_lg,Data_intermediate])
    Data_lg=Data_lg[cols_lg]
    
Data_lg=Data_lg[[x for x in Data_lg.columns if x!='drop']]
Data_lg.index=range(Data_lg.shape[0])

############################################

Data_bid=Data_lg[[x for x in cols_lg if x!='Yield' and x!='drop']].rename(columns={'BidPrice': 'Yield'})
Data_bid['Carrier']='BID'
Data_lg=Data_lg[[x for x in cols_lg if x!='BidPrice' and x!='drop']]

############################################

files_comp=[x for x in os.listdir() if '_comp_' in x and 'business' not in x]
cols_comp=['Origin','Destination','Carrier','Date','Time','drop','Yield','Customer','Direction','Interval']
Data_comp=DataFrame(columns=cols_comp)

for f in files_comp:

    Data_intermediate=pd.read_csv(f,header=None)

    Data_intermediate.columns=['Origin','Destination','Carrier','Date','Time','drop','Yield']

    Data_intermediate['Customer']=f.split('_')[4]
    Data_intermediate['Direction']=f.split('_')[5]
    Data_intermediate['Interval']=f.split('_')[6]
    Data_intermediate=Data_intermediate[cols_comp]

    Data_comp=pd.concat([Data_comp,Data_intermediate])
    Data_comp=Data_comp[cols_comp]
    
Data_comp=Data_comp[[x for x in Data_comp.columns if x!='drop']]
Data_comp.index=range(Data_comp.shape[0])

############################################

Data=pd.concat([Data_lg,Data_comp,Data_bid])

Data['Date']=pd.to_datetime(Data['Date'])

Data['Interval']=Data['Interval'].astype('int')

# Dash

In [1010]:
app = dash.Dash(__name__)

In [1011]:
intermediate=Data[['Origin','Destination','Time','Customer','Direction','Interval']].drop_duplicates()
checkbox_values=intermediate\
.sort_values(by=['Origin','Destination','Time','Customer','Direction','Interval']).transpose().values.tolist()

In [1012]:
mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjajNlcnh3MzEwMHZtMzNueGw3NWw5ZXF5In0.fk8k06T96Ml9CLGgKmk81w'

In [1013]:
layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(
        l=30,
        r=30,
        b=20,
        t=40
    ),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation='h'),
    title='Satellite Overview',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style="light",
        center=dict(
            lon=-78.05,
            lat=42.54
        ),
        zoom=7,
    )
)

In [1014]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [1015]:
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int((dt - epoch).total_seconds())#* 1000.0

def get_marks_from_start_end(start, end):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    result = []
    current = start
    while current <= end:
        result.append(current)
        current += relativedelta(hours=1)
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in result}

def get_marks_from_start_end_2(seq):
    ''' Returns dict with one item per month
    {1440080188.1900003: '2015-08',
    '''
    return {unix_time_millis(m):(str(m.strftime('%H:%M'))) for m in seq}

In [1016]:
# all_marks_out=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
# all_marks_in=get_marks_from_start_end(min(checkbox_values[3]),max(checkbox_values[3]))

# marks_lg=get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates())
# all_marks=dict(zip([x for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000],
# [all_marks[x] for x in all_marks.keys() if min([abs(x-y) for y in marks_lg.keys()])>5000]))
# all_marks.update(marks_lg)

#all_marks=get_marks_from_start_end(min(checkbox_values[2]),max(checkbox_values[2]))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))
#all_marks.update(get_marks_from_start_end_2(Data.loc[Data['carrier']=='LG','outbound_departure_time'].drop_duplicates()))

In [1017]:
PAGE_SIZE = 20

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        html.Div([html.Div(style={'width': '10%', 'display': 'inline-block'}),
        html.Img(src=app.get_asset_url('LuxairGroup.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle', 
                       'width': '10%', 'height': '7em'}),
        html.H4(children='Price Monitoring System',style={
            'textAlign': 'center',
            'color': 'black','height': '1em', 'display': 'inline-block', 'width': '64%'}),
            html.Img(src=app.get_asset_url('Luxair.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle',
                       'width': '10%', 'height': '7em'}),
        html.Div(style={'width': '10%', 'display': 'inline-block', 'vertical-align': 'middle'})
                 ],
        style={'margin-bottom': '0em'})
     ,
    # ROUTE DROPDOWN AND DATE SLIDER
    
        # DROPDOWN
    html.Div([html.Div(style={'width': '10%', 'margin-bottom': '1em', 'display': 'inline-block'}),
    
    html.Div([html.Div([html.Div([html.Label('Origin', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_origin',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[0]))],
    placeholder="Select origin", 
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '4em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Destination', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_destination',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[1]))],
    placeholder="Select destination",
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '2em', 'display': 'inline-block'}),
    html.Div([html.Label('Time', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_time',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[2]))],
    placeholder="Select time",
    value='',
    multi=False)],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '2em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '1em'}),
    html.Div([html.Div([html.Label('Customer Segment', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.RadioItems(id = 'radio_customer',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[3]))],
    labelStyle={'display': 'inline-block'},
    value='')],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '15em', 'display': 'inline-block'})
    ,
    html.Div([html.Label('Direction', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.RadioItems(id = 'radio_direction',
    options=[{'label': i,'value': i} for i in sorted(set(checkbox_values[4]))],
    labelStyle={'display': 'inline-block'},
    value='')],
    style={'margin-bottom': '0.5em', 'width': '30%', 'margin-left': '3em', 'display': 'inline-block'})],          
    style={'margin-bottom': '0.5em', 'width': '90%', 'margin-left': '1em'})
    ,
    html.Div([html.Label('Interval', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Slider(
    min=0,
    max=np.sqrt(1440),
    marks=dict(zip([np.sqrt(x) for x in sorted(intermediate['Interval'].unique())],
    [str(x)+'m' if x<=60 else str(int(round(x/60)))+'h' for x in sorted(intermediate['Interval'].unique())])),
    step=None
    )],
    style={'margin_top': '0em', 'margin-bottom': '5em', 'width': '60%', 'margin-left': '20em'})],
    style={'width': '75%', 'margin-bottom': '1em', 'margin-top': '0','display': 'inline-block', 'background-color': 'lightgrey'}),
    
    html.Div(style={'width': '10%', 'margin-bottom': '0.5em', 'margin-top': '-3em', 'display': 'inline-block'})
             ]),
    # GRAPH CONTAINER                   
    html.Div([html.Div([dcc.Graph(
                    id='graph-outbound')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'}),
    html.Div([dcc.Graph(
                    id='graph-inbound')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'})
             ])
        ])

In [1018]:
@app.callback(
    Output("dropdown_destination", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_time", "value"), 
    Input("radio_customer", "value"),
    Input("radio_direction", "value")]
    )
def set_dropdown_origin(values_origin,values_time,values_customer,values_direction):
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[1]))\
    if checkbox_values[0][i]==values_origin])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

@app.callback(
    Output("dropdown_time", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("radio_customer", "value"),
    Input("radio_direction", "value")]
    )
def set_dropdown_time(values_origin,values_destination,values_customer,values_direction):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i]==values_origin\
    and checkbox_values[1][i]==values_destination])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

##########################################################

@app.callback(
    Output("radio_customer", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"), 
    Input("dropdown_time", "value"),
    Input("radio_direction", "value")]
    )
def set_radio_customer(values_origin,values_destination,values_time,values_direction):
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
    if checkbox_values[0][i]==values_origin\
    and checkbox_values[1][i]==values_destination\
    and checkbox_values[2][i]==values_time])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

In [1019]:
if __name__ == '__main__':
    app.run_server(debug=False,host='10.0.0.3',port=4000)
    #app.run_server(debug=False,host='localhost',port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://10.0.0.3:4000/ (Press CTRL+C to quit)
172.27.2.85 - - [05/Jul/2019 15:33:43] "GET / HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:44] "GET /_dash-dependencies HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:44] "GET /_dash-layout HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:44] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:44] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:44] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:46] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:46] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:46] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:48] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:48] "POST /_dash-update-component HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:33:48] "POST 

In [ ]:
@app.callback(
    Output("dropdown_outtime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value"),
    Input("dropdown_intime", "value")]
    )
def set_checklist_outtime(values_route,values_stay,values_intime):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i] in values_route\
    and checkbox_values[1][i] in values_stay])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

@app.callback(
    Output("dropdown_intime", "options"),
    [Input("dropdown_route", "value"),
    Input("dropdown_stay", "value"),
    Input("dropdown_outtime", "value")]
    )
def set_checklist_intime(values_route,values_stay,values_outtime):
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
    if checkbox_values[0][i] in values_route\
    and checkbox_values[1][i] in values_stay\
    and checkbox_values[2][i] in values_outtime])   
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values
##########################################################

In [ ]:
colors_carrier=dict(zip(['TP','FR','U2','AZ','OS','KL','SK','V7'],
                        ['#71BF44','#f1c933','#FF7D1C','#2c692c','#E71C23','#00a1e4','#2B0F73','#EB2226']))

In [ ]:
@app.callback(Output('graph-inbound', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_route', 'value'),
    Input('dropdown_stay', 'value'),
    Input('dropdown_outtime', 'value'),
    Input('dropdown_intime', 'value')]
    )
def update_plot(value_route, value_stay, value_outtime, value_intime):
    
    dff_plot = Data.loc[(Data['route']==value_route)\
                   &(Data['min_stay']==value_stay)\
                   &(Data['outbound_departure_time_lg']==value_outtime)\
                   &(Data['inbound_departure_time_lg']==value_intime),:]
    
    dff_plot_2=dff_plot.drop_duplicates(subset=['route','min_stay','outbound_departure_time_lg',
                                               'inbound_departure_time_lg','outbound_departure_date'])
    
    carriers=dff_plot['carrier'].unique()
    
    traces = []

    traces.append(go.Scatter(
    x=dff_plot_2['outbound_departure_date'].tolist(),
    y=dff_plot_2['price_outbound_lg'].tolist(),
    mode = 'lines',
    name = 'LG',
    line = dict(
    dash = 'solid',
    color ='blue',
    width = 2
                 )
            ))    

    for j in carriers:
        traces.append(go.Scatter(
        x=dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_date'].tolist(),
        y=dff_plot.loc[dff_plot['carrier']==j,'price_outbound_comp'].tolist(),
        mode = 'lines',
        name = j,
        #+'/'+dff_plot.loc[dff_plot['carrier']==j,'outbound_departure_time_comp'].unique().tolist()[0],
        line = dict(
        dash = 'solid',
        color =colors_carrier[j],
        width = 2
                 )
            ))

    traces.append(go.Scatter(
    x=dff_plot_2['outbound_departure_date'].tolist(),
    y=dff_plot_2['BidPrice_outbound'].tolist(),
    mode = 'lines',
    name = 'Bid Price',
    line = dict(
    dash = 'solid',
    color ='black',
    width = 2
                 )
            ))         
   
    return {'data': traces, 'layout': 
    go.Layout(title=go.layout.Title(
    text='Outbound Flight',
    xref='paper',
    x=0.5),
    xaxis={'title': 'Departure Date'},
    yaxis={'title': 'Yield', 'range': [0,max([Data['price_outbound_lg'].max(),Data['price_outbound_comp'].max()])]},
    height=600)}         

In [ ]:
@app.callback(Output('graph-outbound', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_route', 'value'),
    Input('dropdown_stay', 'value'),
    Input('dropdown_outtime', 'value'),
    Input('dropdown_intime', 'value')]
    )
def update_plot(value_route, value_stay, value_outtime, value_intime):
    
    dff_plot = Data.loc[(Data['route']==value_route)\
                   &(Data['min_stay']==value_stay)\
                   &(Data['outbound_departure_time_lg']==value_outtime)\
                   &(Data['inbound_departure_time_lg']==value_intime),:]
    
    dff_plot_2=dff_plot.drop_duplicates(subset=['route','min_stay','outbound_departure_time_lg',
                                               'inbound_departure_time_lg','outbound_departure_date'])
    
    carriers=dff_plot['carrier'].unique()
    
    traces = []

    traces.append(go.Scatter(
    x=dff_plot_2['inbound_departure_date'].tolist(),
    y=dff_plot_2['price_inbound_lg'].tolist(),
    mode = 'lines',
    name = 'LG',
    line = dict(
    dash = 'solid',
    color ='blue',
    width = 2
                 )
            ))    

    for j in carriers:
        traces.append(go.Scatter(
        x=dff_plot.loc[dff_plot['carrier']==j,'inbound_departure_date'].tolist(),
        y=dff_plot.loc[dff_plot['carrier']==j,'price_inbound_comp'].tolist(),
        mode = 'lines',
        name = j,
        #+'/'+dff_plot.loc[dff_plot['carrier']==j,'inbound_departure_time_comp'].unique().tolist()[0],
        line = dict(
        dash = 'solid',
        color =colors_carrier[j],
        width = 2
                 )
            ))

    traces.append(go.Scatter(
    x=dff_plot_2['inbound_departure_date'].tolist(),
    y=dff_plot_2['BidPrice_inbound'].tolist(),
    mode = 'lines',
    name = 'Bid Price',
    line = dict(
    dash = 'solid',
    color ='black',
    width = 2
                 )
            ))          
        
    return {'data': traces, 'layout': 
    go.Layout(title=go.layout.Title(
    text='Inbound Flight',
    xref='paper',
    x=0.5),
    xaxis={'title': 'Departure Date'},
    yaxis={'title': 'Yield', 'range': [0,max([Data['price_inbound_lg'].max(),Data['price_inbound_comp'].max()])]},
    height=600)}         

In [876]:
if __name__ == '__main__':
    app.run_server(debug=False,host='10.0.0.3',port=4000)
    #app.run_server(debug=False,host='localhost',port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://10.0.0.3:4000/ (Press CTRL+C to quit)
172.27.2.85 - - [05/Jul/2019 15:19:26] "GET / HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:19:26] "GET /_dash-layout HTTP/1.1" 200 -
172.27.2.85 - - [05/Jul/2019 15:19:26] "GET /_dash-dependencies HTTP/1.1" 200 -
